In [111]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models

In [112]:
hc = 12.39841875 #keV*Angström
d_si = 3.1358

In [130]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'

In [131]:
pc_frederik = ''

In [132]:
var_pc = pc_henrik

In [133]:
cd $var_pc/Messdaten

/home/arrow13/Git/FP/Roentgen/Messdaten


#Implementirung der Funktionen für die Auswertung

## Die Peak sollen mit der Gaussverteilung angefitten werde, diese wird hier implementier, wie ein wrappen für automatische fits

# Gauss
<h1>$f(x) = \frac{a}{\sqrt{2 \cdot pi \cdot \sigma^2}} \cdot e^{- \frac{{(x- \sigma )}^2}{2 \cdot \sigma^2}} $ </h1>

In [134]:
def gauss(x,a,mu,sigma):
    return (a/(np.sqrt(2*3.14*sigma**2))*np.exp(-((x-mu)**2)/(2.*sigma**2)))

# ${\chi_{red}}^2$ / Goodness of a fit
<h1> ${\chi_{red}}^2 = \frac{1}{\nu}\sum\frac{{(O-E)}^2}{\sigma^2}$ 

In [150]:
def red_chi_gauss(data, model, deg):
    chisq = np.sum(np.square(data-model)/np.power(data,0.5))
    nu = data.size-1-deg    # -4: -1 for the equation and -3 from the degrees of freedom
    if nu is 0:
        return "err: nu = 0"
    return chisq/nu

In [151]:
def fit_gauss(x,data,guess=[1,1,1]):
    '''
        returns the parameter from the fit, the standard deviation and the reduced chisquare
    '''
    para, cov = opt.curve_fit(gauss,x,np.sqrt(data),guess,maxfev=10000)
    red_chi = red_chi_gauss(data,gauss(x,para[0],para[1],para[2]),3)
    return para, np.sqrt(np.diag(cov)), red_chi

# Bragg-reflexion
$n \cdot \lambda = 2 \cdot d \cdot sin(\theta)$

# Energie
$E = \frac{h \cdot c}{2 \cdot d \cdot sin(\theta)}$

In [152]:
def energie(phi,delta_phi):
    theta = phi/2
    e = hc/(2*d_si*np.sin(theta*(3.14/180)))
    delta_e = (e*delta_phi/2)/np.tan(theta*(3.14/180))
    return e, delta_e

In [153]:
def intensität(a,delta_a,sigma,delta_sigma):
    i = a/(np.sqrt(2*3.14*sigma**2))
    delta_i = np.sqrt((delta_a/np.sqrt(2*3.14*sigma**2))**2 + ((a*delta_sigma)/(np.sqrt(2*3.14*sigma**4)))**2)
    return i, delta_i

In [154]:
def d(e,delta_e,phi,delta_phi,n):
    d = (n*hc)/(2*e*np.sin((phi/2)*(3.14/180)))
    delta_d = d*np.sqrt(((delta_phi/2)/np.tan((phi/2)*(3.14/180)))**2+(delta_e/e)**2)

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = ??, A$_1$ = ??
+ U$_2$ = ??, A$_2$ = ??
+ U$_3$ = ??, A$_3$ = ??

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdefraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

# Ersterwert: Spannung
# Zweiterwert: Anodenstrom


# Vorbereitung für den Voigt-Fit

In [155]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')

In [156]:
data = np.loadtxt("Si111_40_30.xy").T
plt.plot(data[0],data[1])
plt.show()

In [157]:
params_1 = voigt_1.guess(data[1],x=data[0])
params_2 = voigt_2.guess(data[1],x=data[0])
params_3 = voigt_3.guess(data[1],x=data[0])

params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [158]:
fit_1 = voigt_1.fit(data[1],params_1,x=data[0])
fit_2 = voigt_2.fit(data[1],params_2,x=data[0])
fit_3 = voigt_3.fit(data[1],params_3,x=data[0])

In [159]:
peak_1_range = np.arange(np.where( 25.5 < data[0])[0][0],np.where( data[0] > 25.8)[0][0])
peak_2_range = np.arange(np.where( 28.39 < data[0])[0][0],np.where( data[0] > 28.48)[0][0])
peak_3_range = np.arange(np.where( 28.47 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])

In [269]:
params_1 = voigt_1.guess(data[1][peak_1_range],x=data[0][peak_1_range])
params_2 = voigt_2.guess(data[1][peak_2_range],x=data[0][peak_2_range])
params_3 = voigt_3.guess(data[1][peak_3_range],x=data[0][peak_3_range])

params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [270]:
fit_1 = voigt_1.fit(data[1][peak_1_range],params_1,x=data[0][peak_1_range],weights=1/np.sqrt(data[1][peak_1_range]*100))
fit_2 = voigt_2.fit(data[1][peak_2_range],params_2,x=data[0][peak_2_range],weights=1/np.sqrt(data[1][peak_2_range]*100))
fit_3 = voigt_3.fit(data[1][peak_3_range],params_3,x=data[0][peak_3_range],weights=1/np.sqrt(data[1][peak_3_range]*100))

In [271]:
fit_1.plot()
fit_2.plot()
fit_3.plot()
plt.show()

In [166]:
print(fit_1.fit_report())
print(fit_2.fit_report())
print(fit_3.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 317
    # data points      = 71
    # variables        = 4
    chi-square         = 67.844
    reduced chi-square = 1.013
[[Variables]]
    voigt_1fwhm:        1.6381e-11 +/- 0        (0.00%)  == '3.6013100*voigt_1sigma'
    voigt_1center:      25.6460660 +/- 0        (0.00%) (init= 25.64617)
    voigt_1gamma:       0.01685735 +/- 0        (0.00%) (init= 1)
    voigt_1amplitude:   1185.14981 +/- 0        (0.00%) (init= 1214.115)
    voigt_1sigma:       4.5486e-12 +/- 0        (0.00%) (init= 0.01357791)
[[Correlations]] (unreported correlations are <  0.100)
[[Model]]
    Model(voigt, prefix='voigt_2')
[[Fit Statistics]]
    # function evals   = 259
    # data points      = 22
    # variables        = 4
    chi-square         = 45.839
    reduced chi-square = 2.547
[[Variables]]
    voigt_2amplitude:   2537.73244 +/- 408.3406 (16.09%) (init= 1946.006)
    voigt_2sigma:       0.02190427 +/- 0.007294 

In [165]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.plot(data[0][peak_1_range],fit_1.best_fit)
plt.plot(data[0][peak_2_range],fit_2.best_fit)
plt.plot(data[0][peak_3_range],fit_3.best_fit)
plt.show()

In [268]:
multivoigt = voigt_1 + voigt_2 + voigt_3

In [200]:
params = voigt_1.guess(data[1][peak_1_range],x=data[0][peak_1_range])

In [201]:
params.update(voigt_2.guess(data[1][peak_2_range],x=data[0][peak_2_range]))

In [202]:
params.update(voigt_3.guess(data[1][peak_3_range],x=data[0][peak_3_range]))

In [278]:
fit_4 = multivoigt.fit(data[1],params,x=data[0])

In [279]:
fit_4.best_values

{'voigt_1amplitude': 1173.37677030752,
 'voigt_1center': 25.645435196600626,
 'voigt_1gamma': 0.012001902500348383,
 'voigt_1sigma': 0.012001902500348383,
 'voigt_2amplitude': 2579.7109976809411,
 'voigt_2center': 28.433413387296806,
 'voigt_2gamma': 0.014865878353797601,
 'voigt_2sigma': 0.014865878353797601,
 'voigt_3amplitude': 1649.1495740005764,
 'voigt_3center': 28.508399293577501,
 'voigt_3gamma': 0.011943775027492443,
 'voigt_3sigma': 0.011943775027492443}

In [282]:
print(fit_4.fit_report())

[[Model]]
    ((Model(voigt, prefix='voigt_1') + Model(voigt, prefix='voigt_2')) + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 41
    # data points      = 27526
    # variables        = 9
    chi-square         = 1229308139.899
    reduced chi-square = 44674.497
[[Variables]]
    voigt_1fwhm:        0.04322257 +/- 0.000272 (0.63%)  == '3.6013100*voigt_1sigma'
    voigt_1center:      25.6454352 +/- 0.000104 (0.00%) (init= 25.64617)
    voigt_1gamma:       0.01200190 +/- 7.56e-05 (0.63%)  == 'voigt_1sigma'
    voigt_1amplitude:   1173.37677 +/- 5.675885 (0.48%) (init= 1214.115)
    voigt_1sigma:       0.01200190 +/- 7.56e-05 (0.63%) (init= 0.01357791)
    voigt_2amplitude:   2579.71099 +/- 7.980843 (0.31%) (init= 1946.006)
    voigt_2sigma:       0.01486587 +/- 5.62e-05 (0.38%) (init= 0.0149357)
    voigt_2gamma:       0.01486587 +/- 5.62e-05 (0.38%)  == 'voigt_2sigma'
    voigt_2center:      28.4334133 +/- 7.10e-05 (0.00%) (init= 28.43486)
    voigt_2fwhm

In [272]:
fit_4.plot()
plt.show()